In [1]:
pip install opencv-python numpy

In [2]:
pip install opencv-python numpy

In [3]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     -------------------------------------- 0.1/100.1 MB 353.1 kB/s eta 0:04:44
     -------------------------------------- 0.1/100.1 MB 353.1 kB/s eta 0:04:44
     -------------------------------------- 0.1/100.1 MB 353.1 kB/s eta 0:04:44
     -------------------------------------- 0.1/100.1 MB 353.1 kB/s eta 0:04:44
     -------------------------------------- 0.2/100.1 MB 402.6 kB/s eta 0:04:09
     -------------------------------------- 0.2/100.1 MB 402.6 kB/s eta 0:04:09
     -------------------------------------- 0.2/100.1 MB 401.2 kB/s eta 0:04:10

In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# Specify the path to your directory containing known faces
KNOWN_FACES_DIR = r'C:\Nandy'

images = []
class_names = []

# Load known faces
print("Loading known faces...")
for filename in os.listdir(KNOWN_FACES_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(KNOWN_FACES_DIR, filename)
        img = face_recognition.load_image_file(img_path)
        if img is not None:
            encoding = face_recognition.face_encodings(img)
            if len(encoding) > 0:
                images.append(encoding[0])
                class_names.append(os.path.splitext(filename)[0])
            else:
                print(f"Warning: No face found in {filename}")
        else:
            print(f"Warning: Could not read image {img_path}")

if not images:
    print("No valid images found.")
    print("Please provide a directory containing image files of known faces.")
    exit()

print(f"Loaded {len(images)} known faces.")

def mark_attendance(name):
    with open('attendance.csv', 'a+') as f:
        f.seek(0)
        my_data_list = f.readlines()
        name_list = [line.split(',')[0] for line in my_data_list]
        if name not in name_list:
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dt_string}')

# Initialize webcam
print("Attempting to open webcam...")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    print("Available cameras:")
    for i in range(10):  # Check first 10 camera indices
        temp_cap = cv2.VideoCapture(i)
        if temp_cap.isOpened():
            print(f"Camera index {i} is available")
            temp_cap.release()
    exit()

print("Webcam opened successfully.")

frame_count = 0
max_attempts = 50  # Attempt to read up to 50 frames before giving up

while frame_count < max_attempts:
    ret, img = cap.read()
    if not ret:
        print(f"Failed to grab frame. Attempt {frame_count + 1}/{max_attempts}")
        frame_count += 1
        continue
    
    print("Successfully grabbed a frame.")
    
    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(img)
    face_encodings = face_recognition.face_encodings(img, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare the face with known faces
        matches = face_recognition.compare_faces(images, face_encoding)
        name = "Unknown"
        color = (0, 0, 255)  # Red for unknown faces

        if True in matches:
            first_match_index = matches.index(True)
            name = class_names[first_match_index].upper()
            color = (0, 255, 0)  # Green for known faces
            mark_attendance(name)

        cv2.rectangle(img, (left, top), (right, bottom), color, 2)
        cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

if frame_count >= max_attempts:
    print("Failed to grab frames after multiple attempts. Please check your webcam connection.")

cap.release()
cv2.destroyAllWindows()

Loading known faces...
Loaded 5 known faces.
Attempting to open webcam...
Webcam opened successfully.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame.
Successfully grabbed a frame